<center><img src="img/skillfactorylogo.png"></center>

<h1><center>Курс "Практический Machine Learning"</center></h1>
<h3><center>Шестаков Андрей</center></h3>
<hr>
<h2><center>Введение в рекомендательные системы</center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact, IntSlider, fixed, FloatSlider

# Мотивация

* Люди - потребители контента и услуг
    * Музыка
    * Фильмы
    * Книги
    * Игры
    * Еда
    * ...


* Но выбор слишком велик..
    * Spotify - 30 млн. песен
    * Netflix - 20 тыс. фильмов
    * Amazon - 500 тыс. книг
    * Steam - 20 тыс. игр

Надо как-то фильтровать..
* Можно спросить у друзей (вкусы могут отличаться)
* Можно почитать обзоры (много времени)
* Автоматическая рекомендательная система!

<center><img src="img/recsys.jpg" width=400></center>



### Netflix Prize
<center><img src="img/netflix.jpg" width=500></center>

### Источники персональных рекомендаций
* На основе предпочтений пользователя
    * Рассчитывается некоторый "профиль" пользователя, для которого определяются наиболее подходящие товары
* На основе похожих пользователей
    * Находим других пользователей с похожими интересами и доставляем рекомендацию
* На основе похожих товоров
    * Рекомендуем товары, похожие на те, что мне нравятся

# Постановка проблемы

* Пользователи ставят оценку товарам
    * Бинарную
    * Количество "звезд"
    * Неявную (кол-во потраченого времени\денег)
<center><img src="img/rating.png" width=600></center>
* Надо заполнить пропуски
* Предоставить рекомендацию



### Нюансы
* Хорошее восстановление рейтингов $\neq$ хорошая рекомендательная система
* Учет экономических предпочтений продавцов
* Learning Loop
* Холодный старт
* Возникает для новых товаров и пользователей
* Масштабируемость
* Накручивание рейтингов
* Неактивные пользователи
* Тривиальные рекомендации

<center><img src="img/hp-rec.png" width=600></center>

## Подходы к решению
* Коллаборативная фильтрация
* Латентные методы (матричные разложения)

# Коллаборативная фильтрация

* User-based
* Item-based

## User-based CF
<center><img src="img/ub-collab.png" width=700></center>

Введем обозначения:
* $U$ - множество пользователей
* $I$ - множество товаров
* $U_i$ - множество пользователей, оценивших товар $i$
* $I_u$ - множество товаров, оценненных пользователем $u$
* $R_{ui}$ - оценка, которую дал пользователь $u$ товару $i$
* $\hat{R}_{ui}$ - прогноз оценки



### Прогнозирование рейтинга
* Посчитаем сходство между пользователями $s \in \mathbb{R}^{U \times U}$
* Для целевого пользователя $u$ найти похожих пользователей $N(u)$
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv}(R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$

* $\bar{R}_u$ - поправка на писсимизм\оптимизм пользователей

### Как определять $N(u)$?
* $N(u)$ можно определять по разным соображениям:
    * Брать всех
    * Top-k
    * $s_{uv} > \theta$

### Как определять схожесть пользователей

* Для каждой пары $(u,v)$ надо пересечь множество оцененных товаров

* Корреляция пирсона
$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} (R_{ui} - \bar{R}_u)(R_{vi} - \bar{R}_v)}{\sqrt{\sum\limits_{i \in I_u\cap I_v}(R_{ui} - \bar{R}_u)^2}\sqrt{\sum\limits_{i \in I_u\cap I_v}(R_{vi} - \bar{R}_v)^2}}$$
* Корреляция Спирмана
* Косинусная мера
$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$


### Пример

<img src='img/example1.png'>

<img src='img/example2.png'>

## Item-based CF

<center><img src='http://dataconomy.com/wp-content/uploads/2015/03/Beginners-Guide-Recommender-Systems-Content-Based-Filtering.png' width=350></center>

### Прогнозирование рейтинга
* Посчитаем сходство между товарами $s \in \mathbb{R}^{I \times I}$
* Для товара $i$ найти оцененные пользователем $u$ похожие товары: $N(i)$

$$ \hat{R}_{ui} = \frac{\sum_{j \in N(i)} s_{ij}R_{uj}}{\sum_{j \in N(i)} \left| s_{ij}\right|} $$

### Схожесть товаров

* Условная вероятность
$$ s_{ij} = \frac{n_{ij}}{n_i} $$
* Зависимость
$$ s_{ij} = \frac{n_{ij}}{n_i n_j} $$

Попробуем что-то сделать с модулем [surprice](http://surprise.readthedocs.io/en/stable/index.html)

## Демо CF

In [3]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [4]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,1,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [6]:
df_movies.loc[:, 'id'] = df_movies.loc[:, 'id'].astype('str')
df_movies = df_movies.set_index('id')

In [7]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,75,3,1.0,29,10,2006,23,17,16
1,75,32,4.5,29,10,2006,23,23,44
2,75,110,4.0,29,10,2006,23,30,8
3,75,160,2.0,29,10,2006,23,16,52
4,75,163,4.0,29,10,2006,23,29,30


In [10]:
#преобразуем три столбца в дату
df_rates = df_rates.rename(columns={'date_day': 'day',
                                   'date_month': 'month',
                                   'date_year': 'year'})
df_rates.loc[:,'datetime'] = pd.to_datetime(df_rates.loc[:,['year', 'month', 'day']])

In [11]:
q = df_rates.datetime.quantile(0.85)

In [13]:
q

Timestamp('2008-02-01 00:00:00')

In [14]:
filepath = './data/user_ratedmovies_train.dat'
idx = df_rates.datetime < q
df_rates.loc[idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

filepath = './data/user_ratedmovies_test.dat'
df_rates.loc[~idx].to_csv(filepath, sep='\t', columns=['userID', 'movieID', 'rating'], index=None)

In [17]:
from surprise import Dataset, Reader

In [18]:
filepaths = [('./data/user_ratedmovies_train.dat', './data/user_ratedmovies_test.dat')]
reader = Reader(line_format='user item rating', sep='\t', skip_lines=1)
data = Dataset.load_from_folds(filepaths, reader=reader)

In [19]:
#метод коллаборативной фильтрации
from surprise import KNNBasic, KNNWithMeans
from surprise.accuracy import rmse
from surprise import dump

Описание алгоритмов, основанных на CF - [туть](http://surprise.readthedocs.io/en/stable/knn_inspired.html)

In [20]:
sim_options = {'name': 'cosine',
               'user_based': True
               }

In [21]:
dumpfile = './alg.dump'

In [23]:
algo = KNNWithMeans(k=20, min_k=1, sim_options=sim_options)                                                       

for trainset, testset in data.folds(): 
    algo.train(trainset)                             
    predictions = algo.test(testset)
    rmse(predictions)
    
    dump.dump(dumpfile, predictions, algo)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9221


In [24]:
df_predictions = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details']) 

In [25]:
df_predictions.head()

,uid,iid,rui,est,details
0,190,173,1.0,2.281462,"{'actual_k': 20, 'was_impossible': False}"
1,190,367,3.0,2.521914,"{'actual_k': 20, 'was_impossible': False}"
2,190,368,3.0,2.876713,"{'actual_k': 20, 'was_impossible': False}"
3,190,380,2.5,2.979809,"{'actual_k': 20, 'was_impossible': False}"
4,190,480,4.5,2.915144,"{'actual_k': 20, 'was_impossible': False}"


In [26]:
algo.predict('190', '173', verbose=2)

user: 190        item: 173        r_ui = None   est = 2.28   {'actual_k': 20, 'was_impossible': False}


Prediction(uid='190', iid='173', r_ui=None, est=2.2814620549058646, details={'actual_k': 20, 'was_impossible': False})

In [27]:
anti_train = trainset.build_anti_testset()

In [28]:
one_user = filter(lambda r: r[0] == '75', anti_train)

In [29]:
# Это будет долго..
# anti_train_predictions = algo.test(one_user)

anti_train_predictions = algo.test(one_user)

In [30]:
from collections import defaultdict

def get_top_n(predictions, n=10):

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [33]:
df_rates.head()

,userID,movieID,rating,day,month,year,date_hour,date_minute,date_second,datetime
0,75,3,1.0,29,10,2006,23,17,16,2006-10-29
1,75,32,4.5,29,10,2006,23,23,44,2006-10-29
2,75,110,4.0,29,10,2006,23,30,8,2006-10-29
3,75,160,2.0,29,10,2006,23,16,52,2006-10-29
4,75,163,4.0,29,10,2006,23,29,30,2006-10-29


In [31]:
df_movies.loc['5695', 'title']

'To the Devil a Daughter'

In [32]:
top_n = get_top_n(anti_train_predictions, n=10)

for uid, user_ratings in top_n.items():
    print(uid, [df_movies.loc[iid, 'title'] for (iid, _) in user_ratings])

75 ['To the Devil a Daughter', 'Frankenstein and the Monster from Hell', 'La frusta e il corpo', '¿Quién diablos es Juliette?', 'Shu dan long wei', 'Gabbeh', 'Storefront Hitchcock', 'Passion', "La vie et rien d'autre", 'Naturally Native']


# Модели со скрытыми факторами

Для каждого пользователя и товара построим векторы $p_u\in \mathbb{R}^{k}$ и $q_i \in \mathbb{R}^{k}$ так, чтобы
$$ R_{ui} \approx p_u^\top q_i $$

* $p_u$ иногда получается интерпретировать как заинтересованность пользователя в некоторой категории товаров
* $q_i$ иногда получается интерпретировать как принадлежность товара к определенной категории

Кроме того, в полученном пространстве, можно считать похожесть пользователей и товаров

<center><img src='img/matrix_factorization.png' width=600></center>

## Non-negative Matrix Factorization

* $P \geq 0$
* $Q \geq 0$

## SVD разложение
<center><img src='img/svd.png' width=600></center>

* Надо чем-то заполнить пропуски
    * Нулями
    * Базовыми предсказаниями
* Как вариант 
    * $R' = R-B$ и заполнить $0$
* Таким образом:
    * $P = U\sqrt{\Sigma}$
    * $Q = \sqrt{\Sigma}V^\top$
    * $\hat{R} = P^\top Q$
* А как делать предсказания для новых пользователей?

## Non-negative Matrix Factorization

* $P \geq 0$
* $Q \geq 0$

##  Latent Factor Model
* Будем оптимизировать следующий функционал
$$ \sum\limits_{u,i}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle)^2 + \lambda \sum_u\| p_u \|^2 + \mu\sum_i\| q_i \|^2 \rightarrow \min\limits_{P, Q} $$
* С помощью градиентного спуска (на каждом шаге случайно выбирая пару $(u,i)$:
$$ p_{uk} = p_{uk} + 2\alpha \left(q_{ik}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \lambda p_{uk}\right)$$
$$ q_{ik} = q_{ik} + 2\alpha \left(p_{uk}(R_{ui} - \bar{R}_u - \bar{R}_i - \langle p_u, q_i \rangle) - \mu q_{ik}\right)$$

## Демо SVD

### Перекодируем ID фильмов и пользователей

In [34]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

In [35]:
from sklearn.preprocessing import LabelEncoder

In [36]:
mov_enc = LabelEncoder()
mov_enc.fit(df_rates.movieID.values)
n_movies = df_rates.movieID.nunique()

In [37]:
user_enc = LabelEncoder()
user_enc.fit(df_rates.userID.values)
n_users = df_rates.userID.nunique()

In [38]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx, :]

In [39]:
df_rates.loc[:, 'movieID'] = mov_enc.transform(df_rates.movieID.values)
df_movies.loc[:, 'id'] = mov_enc.transform(df_movies.loc[:, 'id'].values)
df_rates.loc[:, 'userID'] = user_enc.transform(df_rates.userID.values)

In [40]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


### В явном виде запишем матрицу рейтингов

In [41]:
from scipy.sparse import coo_matrix, csr_matrix

In [42]:
n_users_train = df_rates.userID.nunique()
R_train = coo_matrix((df_rates.rating, 
                     (df_rates.userID.values, df_rates.movieID.values)),
                     shape=(n_users, n_movies))

In [43]:
from scipy.sparse.linalg import svds

In [44]:
u, s, vt = svds(R_train, k=10, )

In [45]:
vt.shape

(10, 10109)

In [46]:
from sklearn.neighbors import NearestNeighbors

In [47]:
nn = NearestNeighbors(n_neighbors=10, metric='cosine')

In [48]:
v = vt.T

In [49]:
nn.fit(v)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [50]:
ind = nn.kneighbors(v, return_distance=False)

In [51]:
m_names = df_movies.title.values

In [52]:
m_names = pd.DataFrame(data=m_names[ind], columns=['movie']+['nn_{}'.format(i) for i in range(1,10)])

In [53]:
idx = m_names.movie.str.contains('Terminator')
m_names.loc[idx]

,movie,nn_1,nn_2,nn_3,nn_4,nn_5,nn_6,nn_7,nn_8,nn_9
566,Terminator 2: Judgment Day,Terminator Salvation,Die Hard,Indiana Jones and the Last Crusade,Mission: Impossible III,Braveheart,True Lies,Star Wars: Episode VI - Return of the Jedi,Minority Report,Gladiator
1119,Terminator Salvation,Terminator 2: Judgment Day,Die Hard,Aliens,Total Recall,Alien,Waterworld,Indiana Jones and the Last Crusade,True Lies,Batman
6126,Terminator 3: Rise of the Machines,AVP: Alien vs. Predator,Alone in the Dark,Constantine,Battlefield Earth: A Saga of the Year 3000,Predator,Red Planet,AVPR: Aliens vs Predator - Requiem,Judge Dredd,D-War


# Оценка качества

* Качество рейтингов
    * MAE, MSE
* Качество событий
    * F-score, ROC-AUC, PR-AUC
    * precision@k, recall@k